In [ ]:
# 설치
! pip install smolagents
# 마지막 릴리스 대신 소스에서 설치하려면 위 명령을 주석 처리하고 다음 명령의 주석을 해제하십시오.
# ! pip install git+https://github.com/huggingface/smolagents.git

# 에이전트를 사용한 웹 브라우저 자동화 🤖🌐

이 노트북에서는 **에이전트 기반 웹 브라우저 자동화 시스템**을 만들 것입니다! 이 시스템은 웹사이트를 탐색하고, 요소와 상호 작용하며, 정보를 자동으로 추출할 수 있습니다.

에이전트는 다음을 수행할 수 있습니다.

- [x] 웹 페이지로 이동
- [x] 요소 클릭
- [x] 페이지 내 검색
- [x] 팝업 및 모달 처리
- [x] 정보 추출

이 시스템을 단계별로 설정해 봅시다!

먼저 다음 줄을 실행하여 필요한 종속성을 설치하십시오.

```bash
pip install smolagents selenium helium pillow -q
```

필요한 라이브러리를 가져오고 환경 변수를 설정해 봅시다.

In [ ]:
from io import BytesIO
from time import sleep

import helium
from dotenv import load_dotenv
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from smolagents import CodeAgent, tool
from smolagents.agents import ActionStep

# 환경 변수 로드
load_dotenv()

이제 에이전트가 웹 페이지를 탐색하고 상호 작용할 수 있도록 하는 핵심 브라우저 상호 작용 도구를 만들어 보겠습니다.

In [ ]:
@tool
def search_item_ctrl_f(text: str, nth_result: int = 1) -> str:
    """
    Ctrl + F를 통해 현재 페이지에서 텍스트를 검색하고 n번째 항목으로 이동합니다.
    Args:
        text: 검색할 텍스트입니다.
        nth_result: 이동할 항목 번호입니다(기본값: 1).
    """
    elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{text}')]")
    if nth_result > len(elements):
        raise Exception(f"일치 항목 n°{nth_result}을(를) 찾을 수 없습니다({len(elements)}개의 일치 항목만 발견됨).")
    result = f"{len(elements)}개의 '{text}'에 대한 일치 항목을 찾았습니다."
    elem = elements[nth_result - 1]
    driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    result += f"요소 {nth_result}/{len(elements)}에 초점을 맞췄습니다."
    return result

@tool
def go_back() -> None:
    """이전 페이지로 돌아갑니다."""
    driver.back()

@tool
def close_popups() -> str:
    """
    페이지에 표시되는 모든 모달 또는 팝업을 닫습니다. 팝업 창을 닫으려면 이 도구를 사용하십시오!
    이것은 쿠키 동의 배너에서는 작동하지 않습니다.
    """
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()

Chrome으로 브라우저를 설정하고 스크린샷 기능을 구성해 보겠습니다.

In [ ]:
# Chrome 옵션 구성
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--force-device-scale-factor=1")
chrome_options.add_argument("--window-size=1000,1350")
chrome_options.add_argument("--disable-pdf-viewer")
chrome_options.add_argument("--window-position=0,0")

# 브라우저 초기화
driver = helium.start_chrome(headless=False, options=chrome_options)

# 스크린샷 콜백 설정
def save_screenshot(memory_step: ActionStep, agent: CodeAgent) -> None:
    sleep(1.0)  # 스크린샷을 찍기 전에 JavaScript 애니메이션이 발생하도록 합니다.
    driver = helium.get_driver()
    current_step = memory_step.step_number
    if driver is not None:
        for previous_memory_step in agent.memory.steps:  # 간결한 처리를 위해 이전 스크린샷 제거
            if isinstance(previous_memory_step, ActionStep) and previous_memory_step.step_number <= current_step - 2:
                previous_memory_step.observations_images = None
        png_bytes = driver.get_screenshot_as_png()
        image = Image.open(BytesIO(png_bytes))
        print(f"브라우저 스크린샷 캡처: {image.size} 픽셀")
        memory_step.observations_images = [image.copy()]  # 지속되도록 복사본 만들기

    # 현재 URL로 관찰 업데이트
    url_info = f"현재 URL: {driver.current_url}"
    memory_step.observations = (
        url_info if memory_step.observations is None else memory_step.observations + "\n" + url_info
    )

이제 웹 자동화 에이전트를 만들어 보겠습니다.

In [ ]:
from smolagents import InferenceClientModel

# 모델 초기화
model_id = "Qwen/Qwen2-VL-72B-Instruct"  # 선호하는 VLM 모델로 변경할 수 있습니다.
model = InferenceClientModel(model_id=model_id)

# 에이전트 만들기
agent = CodeAgent(
    tools=[go_back, close_popups, search_item_ctrl_f],
    model=model,
    additional_authorized_imports=["helium"],
    step_callbacks=[save_screenshot],
    max_steps=20,
    verbosity_level=2,
)

# 에이전트에 helium 가져오기
agent.python_executor("from helium import *", agent.state)

에이전트에는 웹 자동화에 Helium을 사용하는 방법에 대한 지침이 필요합니다. 다음은 제공할 지침입니다.

In [ ]:
helium_instructions = """
helium을 사용하여 웹사이트에 액세스할 수 있습니다. helium 드라이버에 대해서는 신경 쓰지 마십시오. 이미 관리되고 있습니다.
이미 "from helium import *"를 실행했습니다.
그런 다음 페이지로 이동할 수 있습니다!
코드:
go_to('github.com/trending')
```<end_code>

표시되는 텍스트를 입력하여 클릭 가능한 요소를 직접 클릭할 수 있습니다.
코드:
click("최고 제품")
```<end_code>

링크인 경우:
코드:
click(Link("최고 제품"))
```<end_code>

요소와 상호 작용하려고 할 때 해당 요소를 찾을 수 없으면 LookupError가 발생합니다.
일반적으로 각 버튼 클릭 후 작업을 중지하여 스크린샷에서 어떤 일이 발생하는지 확인하십시오.
페이지에 로그인하려고 하지 마십시오.

위 또는 아래로 스크롤하려면 스크롤할 픽셀 수를 인수로 사용하여 scroll_down 또는 scroll_up을 사용하십시오.
코드:
scroll_down(num_pixels=1200) # 이렇게 하면 뷰포트 하나만큼 아래로 스크롤됩니다.
```<end_code>

닫기 위해 십자 아이콘이 있는 팝업이 있는 경우 해당 요소를 찾거나 'X' 요소를 대상으로 하여 닫기 아이콘을 클릭하려고 하지 마십시오(대부분 실패합니다).
내장된 `close_popups` 도구를 사용하여 닫으십시오.
코드:
close_popups()
```<end_code>

.exists()를 사용하여 요소의 존재 여부를 확인할 수 있습니다. 예를 들면 다음과 같습니다.
코드:
if Text('쿠키 수락?').exists():
    click('수락합니다')
```<end_code>
"""

이제 작업으로 에이전트를 실행할 수 있습니다! Wikipedia에서 정보를 찾아보겠습니다.

In [ ]:
search_request = """
https://en.wikipedia.org/wiki/Chicago로 이동하여 건설 사고를 언급하는 "1992"라는 단어가 포함된 문장을 알려주세요.
"""

agent_output = agent.run(search_request + helium_instructions)
print("최종 출력:")
print(agent_output)

요청을 수정하여 다른 작업을 실행할 수 있습니다. 예를 들어, 제가 더 열심히 일해야 하는지 알아보겠습니다.

In [ ]:
github_request = """
github.com/trending에서 리포지토리를 얻으려면 얼마나 열심히 일해야 하는지 알아보려고 합니다.
최상위 트렌드 리포지토리의 최상위 작성자 프로필로 이동하여 지난 1년 동안의 총 커밋 수를 알려주시겠어요?
"""

agent_output = agent.run(github_request + helium_instructions)
print("최종 출력:")
print(agent_output)

이 시스템은 다음과 같은 작업에 특히 효과적입니다.
- 웹사이트에서 데이터 추출
- 웹 조사 자동화
- UI 테스트 및 검증
- 콘텐츠 모니터링